In [1]:
import random
import pandas as pd
import numpy as np
from pprint import pprint
import sys
sys.path.append('..')
import doctable as dt

## ```tokens``` and ```subdocs``` Custom Types

```tokens``` is used to store a sequence of tokens and ```subdocs``` is used to store a sequence of a sequence of tokens (think list of tokenized sentences). Simply specify these columns as their appropriate type and it will automatically convert these tokenized objects to formatted strings for database storage.

In [2]:
schema = (
    ('id','integer',dict(primary_key=True, autoincrement=True)),
    ('title','string', dict(nullable=False)),
    ('bag_of_words','tokens'),
    ('tokenized_sentences', 'subdocs'),
    ('paragraphs', 'subsubdocs'),
    ('data', 'cpickle'),
)
db = dt.DocTable2(schema)
print(db)

<DocTable2::_documents_ ct: 0>


In [3]:
tokens = ['this', 'is', 'the', 'happiest', 'day', 'of', 'my', 'life', '.']
db.insert({'title':'Happy sentence', 'bag_of_words':tokens})
db.insert({'title':'Happy nothing'})
db.select(['title','bag_of_words'])

[('Happy sentence', ['this', 'is', 'the', 'happiest', 'day', 'of', 'my', 'life', '.']),
 ('Happy nothing', None)]

Under the hood, DocTable2 stores a ```tokens``` column as a set of strings separated by newlines. This is more efficient than storing a pickled list or other sequence because large files can compress text data efficiently.

In [4]:
sentences = (
    ('i', 'am', 'happy','.'),
    ('the','sky','is','blue','.'),
    ('sun', 'is', 'shining','.'),
    ('what', 'more', 'can', 'i', 'ask', 'for', '?'),
)

db.insert({'title':'Happy sentences', 'tokenized_sentences':sentences})
for title,sents in db.select(['title','tokenized_sentences']):
    print(title)
    pprint(sents)
    print()

Happy sentence
None

Happy nothing
None

Happy sentences
(('i', 'am', 'happy', '.'),
 ('the', 'sky', 'is', 'blue', '.'),
 ('sun', 'is', 'shining', '.'),
 ('what', 'more', 'can', 'i', 'ask', 'for', '?'))



In [5]:
paragraphs = (
    (
        ('i', 'am', 'happy','.'),
        ('the','sky','is','blue','.'),
    ),
    (
        ('sun', 'is', 'shining','.'),
        ('what', 'more', 'can', 'i', 'ask', 'for', '?'),
    )
)

db.insert({'title':'Happy paragraphs', 'paragraphs':paragraphs}, ifnotunique='replace')
for title,paragraphs in db.select(['title','paragraphs']):
    print(title)
    pprint(paragraphs)
    print()

Happy sentence
None

Happy nothing
None

Happy sentences
None

Happy paragraphs
((('i', 'am', 'happy', '.'), ('the', 'sky', 'is', 'blue', '.')),
 (('sun', 'is', 'shining', '.'),
  ('what', 'more', 'can', 'i', 'ask', 'for', '?')))



In [6]:
data = {'hello':0, 'world':1}
db.insert({'title':'my data', 'data':data}, ifnotunique='replace')
for title,data in db.select(['title','data']):
    print(title)
    pprint(data)
    print()

Happy sentence
None

Happy nothing
None

Happy sentences
None

Happy paragraphs
None

my data
{'hello': 0, 'world': 1}



In [7]:
db.select_df()

,id,title,bag_of_words,tokenized_sentences,paragraphs,data
0,1,Happy sentence,"[this, is, the, happiest, day, of, my, life, .]",None,None,None
1,2,Happy nothing,None,None,None,None
2,3,Happy sentences,None,"((i, am, happy, .), (the, sky, is, blue, .), (...",None,None
3,4,Happy paragraphs,None,None,"(((i, am, happy, .), (the, sky, is, blue, .)),...",None
4,5,my data,None,None,None,"{'hello': 0, 'world': 1}"
